In [45]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

from const import DATA_DIR

spatial_df = pd.read_csv(DATA_DIR / 'chornobyl/data/1_Spatial_dataset.csv')

# Clean data
map_data = spatial_df.dropna(subset=['Latitude', 'Longitude', '137Cs']).copy()
map_data = map_data[map_data['137Cs'] > 0]

def get_circle_coords(lat, lon, radius_km):
    R = 6378.137 
    theta = np.linspace(0, 2*np.pi, 200)
    d_lat = (radius_km / R) * np.cos(theta)
    d_lon = (radius_km / R) * np.sin(theta) / np.cos(np.radians(lat))
    return lat + np.degrees(d_lat), lon + np.degrees(d_lon)

npp_lat, npp_lon = 51.389, 30.099
center_lat, center_lon = 51.387, 29.80
initial_zoom = 6

fig_map = px.scatter_map(
    map_data,
    lat='Latitude', lon='Longitude',
    color='137Cs', size='137Cs',
    color_continuous_scale='YlOrRd',
    size_max=20, zoom=initial_zoom,
    map_style='carto-darkmatter-nolabels',
    title='<b>The Invisible Footprint: Cesium-137 Soil Contamination</b><br><sub>Spatial distribution of radioactive isotopes in the Chornobyl Exclusion Zone (1997)</sub>',
    labels={'137Cs': 'Cs-137 (Bq/kg)'},
    center=dict(lat=center_lat, lon=center_lon),
    opacity=0.8
)

# Add zones
for i in range(10, 70, 20):
    lats, lons = get_circle_coords(npp_lat, npp_lon, i)
    fig_map.add_trace(go.Scattermap(
        lat=lats, lon=lons, mode='lines',
        line=dict(color='#fff', width=0.5), name='', showlegend=False, hoverinfo='skip',
    ))
    label_angle = 245
    label_lat = npp_lat + (i / 6378.137) * np.cos(np.radians(label_angle)) * 180 / np.pi
    label_lon = npp_lon + (i / 6378.137) * np.sin(np.radians(label_angle)) / np.cos(np.radians(npp_lat)) * 180 / np.pi
    
    fig_map.add_trace(go.Scattermap(
        lat=[label_lat], lon=[label_lon], mode='text+markers',
        marker=dict(size=25, color='#0e0e0e', symbol='circle'),
        name='', showlegend=False, hoverinfo='skip',
        text=[f'{i}km'], textposition='middle center',
        textfont=dict(color='white', size=12),
    ))

fig_map.add_trace(go.Scattermap(
    lat=[npp_lat], lon=[npp_lon], mode='markers+text',
    marker=dict(size=10, color='#fff', symbol='circle'),
    text=['Chornobyl NPP'], textposition="top right",
    name='', showlegend=False, textfont=dict(color='white', size=12),
))

lat_buffer = 0.3365
lon_buffer = 0.9 # slightly wider for aspect ratio

fig_map.update_layout(
    height=600,
    width=1000,
    margin=dict(r=0, t=80, l=0, b=0),
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, font=dict(color="white"), bgcolor="rgba(0,0,0,0.5)"),
    dragmode=False, 
    map=dict(
        center=dict(lat=center_lat, lon=center_lon),
        zoom=initial_zoom,
        bounds=dict(
            west=center_lon - lon_buffer, 
            east=center_lon + lon_buffer, 
            south=center_lat - lat_buffer, 
            north=center_lat + lat_buffer
        )
    )
)

fig_map.show(config={'scrollZoom': True, 'displayModeBar': False})

In [46]:
from const import VISUALIZATIONS_DIR

fig_map.write_html(VISUALIZATIONS_DIR / "the-invisible-footprint.html")